<center><img src="http://sydney.edu.au/images/content/about/logo-mono.jpg"></center>

<center><h1>Statistical Learning and Data Mining (QBUS3820)</h1></center>
<center><h2>Tutorial 12: Artificial Neural Network </h2></center>
<br>


In this tutorial, we will build a spam detector using ANN. To this end, we will build and train a neural network using Keras with a Tensorflow backend. You will learn how to optimise the network using Keras interface to the tensor flow deep learning library. 
In particular, we will apply weight regularization to improve the performance of the deep learning neural network.
Weight regularization provides an approach to reduce the overfitting of the deep neural network on the training data but to improve the performance of the model on new data, such as the holdout test set.

You will need to run the following on a terminal (command prompt on Windows) to install Keras and Tensorflow libraries.

conda install tensorflow

conda install keras

Our workflow proceeds with the following steps.

<a href="#Data-Preparation">Data Preparation</a> <br>
<a href="#Model-Specification">Model Specification</a> <br>
<a href="#Model-Definition">Model Definition</a> <br>
<a href="#Model-Compilation">Model Compilation</a> <br>
<a href="#Fitting-the-model">Fitting the model</a> <br>
<a href="#Model-Prediction">Model Prediction</a> <br>
<a href="#Model-Evaluation">Model Evaluation</a> <br>
<a href="#Some-concluding-remarks">Some-concluding-remarks</a> <br>

This notebook relies on the following libraries and settings.

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l2
import pandas as pd
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


## Data Preparation

Here I received spam.data which is quite difficult to be read correctly. Hence I used R to split the original data set into train and test sets. Then after importing into python, we will preprocess the data so that we can start building your own neural network!

In [2]:
train_data = pd.read_csv('Datasets/spam_train.csv',skiprows=[0],header=None)

In [3]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,4210,0.00,0.00,1.17,0.0,0.00,0.00,0.0,0.00,0.00,...,0.000,0.284,0.0,0.284,0.213,0.071,1.565,12,72,email
1,4311,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,3.024,0.059,0.0,0.059,0.000,0.237,5.016,43,311,email
2,1316,0.27,0.27,0.55,0.0,0.27,0.27,0.0,1.39,0.27,...,0.000,0.279,0.0,2.001,0.093,0.000,3.706,63,341,spam
3,3819,0.00,0.00,0.00,0.0,0.67,0.00,0.0,0.00,0.00,...,0.117,0.117,0.0,0.000,0.000,0.000,1.222,5,33,email
4,2951,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.000,0.000,0.0,0.000,0.000,0.000,1.250,2,10,email


In [4]:
detect={'spam':1,'email':0}

In [5]:
train_data.iloc[:,58] = [detect[item] for item in train_data.iloc[:,58]]
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,4210,0.00,0.00,1.17,0.0,0.00,0.00,0.0,0.00,0.00,...,0.000,0.284,0.0,0.284,0.213,0.071,1.565,12,72,0
1,4311,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,3.024,0.059,0.0,0.059,0.000,0.237,5.016,43,311,0
2,1316,0.27,0.27,0.55,0.0,0.27,0.27,0.0,1.39,0.27,...,0.000,0.279,0.0,2.001,0.093,0.000,3.706,63,341,1
3,3819,0.00,0.00,0.00,0.0,0.67,0.00,0.0,0.00,0.00,...,0.117,0.117,0.0,0.000,0.000,0.000,1.222,5,33,0
4,2951,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.000,0.000,0.0,0.000,0.000,0.000,1.250,2,10,0


In [6]:
train_data.to_csv('train_data.csv',header=False, index=False)

In [7]:
test_data = pd.read_csv('Datasets/spam_test.csv',skiprows=[0],header=None)

In [8]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,3,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,...,0.01,0.143,0.0,0.276,0.184,0.01,9.821,485,2259,spam
1,4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,...,0.00,0.137,0.0,0.137,0.000,0.00,3.537,40,191,spam
2,10,0.06,0.12,0.77,0.0,0.19,0.32,0.38,0.00,0.06,...,0.04,0.030,0.0,0.244,0.081,0.00,1.729,43,749,spam
3,13,0.00,0.69,0.34,0.0,0.34,0.00,0.00,0.00,0.00,...,0.00,0.056,0.0,0.786,0.000,0.00,3.728,61,261,spam
4,21,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.729,0.0,0.729,0.000,0.00,3.833,9,23,spam


In [9]:
test_data.iloc[:,58] = [detect[item] for item in test_data.iloc[:,58]]
test_data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,3,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,...,0.01,0.143,0.0,0.276,0.184,0.01,9.821,485,2259,1
1,4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,...,0.00,0.137,0.0,0.137,0.000,0.00,3.537,40,191,1
2,10,0.06,0.12,0.77,0.0,0.19,0.32,0.38,0.00,0.06,...,0.04,0.030,0.0,0.244,0.081,0.00,1.729,43,749,1
3,13,0.00,0.69,0.34,0.0,0.34,0.00,0.00,0.00,0.00,...,0.00,0.056,0.0,0.786,0.000,0.00,3.728,61,261,1
4,21,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.729,0.0,0.729,0.000,0.00,3.833,9,23,1


In [10]:
test_data.to_csv('test_data.csv',header=False, index=False)

In [11]:
type(test_data.head(1))

pandas.core.frame.DataFrame

## Model Specification

Now we are ready to build our network with Keras. First, we need to specify the **Architecture**- How many layers, how many nodes in each layer, what activation functions do you want to use in each layer. Then create train and test sets.

In [12]:
import pandas as pd
import numpy as np
from keras.models import Sequential  # building NN layer by layer
from keras.layers import Dense # randomly initialise weights to small number close to 0
from keras.regularizers import l2 # reduce the risk of overfitting
from sklearn.metrics import confusion_matrix

training_data = pd.read_csv("train_data.csv")
test_data = pd.read_csv("test_data.csv")  
hidden_size = 2
input_size = 58
output_size = 1
x_train = training_data.iloc[:,0:input_size]
y_train = training_data.iloc[:,input_size:input_size + output_size]
x_test = test_data.iloc[:, 0:input_size]
y_test = test_data.iloc[:, input_size:input_size + output_size]

## Model Definition
Now we create the Keras model.

In [13]:
# Model definition
model = Sequential() # no need to put any parameter as we are defining layer manually
model.add(Dense(hidden_size, activation='relu', input_shape=(input_size,), kernel_regularizer=l2(0.1)))
#model.add(Dense(2, activation='relu', input_shape=(58,)))
#model.add(Dense(6, activation='relu'))
#model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


## Model Compilation
Since spam detection is a classification problem, we care about accuracy, so we include `metrics=['accuracy']`

In [14]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

## Fitting the Model
Here we train our model with our data

In [15]:
model.summary()
# fit model
history = model.fit(x_train, y_train, batch_size=128, verbose=1, epochs=1000)
# Model Evaluation
eval = model.evaluate(x_test, y_test)
print("\nModel Loss: "+str(eval[0]))
print("Model Accuracy: "+str(eval[1]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2)                 118       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
3220/3220 [==============================] - 0s 65us/step - loss: 10.0615 - acc: 0.3885
Epoch 2/1000
3220/3220 [==============================] - 0s 10us/step - loss: 9.9804 - acc: 0.3894
Epoch 3/1000
3220/3220 [==============================] - 0s 10us/step - loss: 9.9192 - acc: 0.3894
Epoch 4/1000
3220/3220 [==============================] - 0s 10us/step - loss: 9.8702 - acc: 0.3894
Epoch 5/1000
3220/3220 [==============================] - 0s 10us/step - loss: 9.8310 - acc

3220/3220 [==============================] - 0s 9us/step - loss: 9.7338 - acc: 0.3894
Epoch 75/1000
3220/3220 [==============================] - 0s 10us/step - loss: 9.7338 - acc: 0.3894
Epoch 76/1000
3220/3220 [==============================] - 0s 9us/step - loss: 9.7338 - acc: 0.3894
Epoch 77/1000
3220/3220 [==============================] - 0s 10us/step - loss: 9.7338 - acc: 0.3894
Epoch 78/1000
3220/3220 [==============================] - 0s 9us/step - loss: 9.7338 - acc: 0.3894
Epoch 79/1000
3220/3220 [==============================] - 0s 9us/step - loss: 9.7338 - acc: 0.3894
Epoch 80/1000
3220/3220 [==============================] - 0s 10us/step - loss: 9.7338 - acc: 0.3894
Epoch 81/1000
3220/3220 [==============================] - 0s 9us/step - loss: 9.7338 - acc: 0.3894
Epoch 82/1000
3220/3220 [==============================] - 0s 9us/step - loss: 9.7338 - acc: 0.3894
Epoch 83/1000
3220/3220 [==============================] - 0s 9us/step - loss: 9.7338 - acc: 0.3894
Epoch 84/10

3220/3220 [==============================] - 0s 10us/step - loss: 0.6634 - acc: 0.3894
Epoch 156/1000
3220/3220 [==============================] - 0s 10us/step - loss: 0.6707 - acc: 0.3894
Epoch 157/1000
3220/3220 [==============================] - 0s 10us/step - loss: 0.6414 - acc: 0.3894
Epoch 158/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.7001 - acc: 0.3894
Epoch 159/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.6142 - acc: 0.3894
Epoch 160/1000
3220/3220 [==============================] - 0s 10us/step - loss: 0.6333 - acc: 0.3894
Epoch 161/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.6321 - acc: 0.3894
Epoch 162/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.6080 - acc: 0.3894
Epoch 163/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.6393 - acc: 0.3894
Epoch 164/1000
3220/3220 [==============================] - 0s 10us/step - loss: 0.5955 - acc: 0.3894


3220/3220 [==============================] - 0s 9us/step - loss: 0.2887 - acc: 0.9211
Epoch 236/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.2583 - acc: 0.9432
Epoch 237/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.3866 - acc: 0.8978
Epoch 238/1000
3220/3220 [==============================] - 0s 8us/step - loss: 0.2594 - acc: 0.9444
Epoch 239/1000
3220/3220 [==============================] - 0s 8us/step - loss: 0.3312 - acc: 0.9062
Epoch 240/1000
3220/3220 [==============================] - 0s 8us/step - loss: 0.2149 - acc: 0.9661
Epoch 241/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.2791 - acc: 0.9289
Epoch 242/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.2605 - acc: 0.9351
Epoch 243/1000
3220/3220 [==============================] - 0s 8us/step - loss: 0.2864 - acc: 0.9208
Epoch 244/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.2693 - acc: 0.9295
Epoch

3220/3220 [==============================] - 0s 9us/step - loss: 0.2278 - acc: 0.9357
Epoch 317/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1847 - acc: 0.9506
Epoch 318/1000
3220/3220 [==============================] - 0s 10us/step - loss: 0.2317 - acc: 0.9335
Epoch 319/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1741 - acc: 0.9534
Epoch 320/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.2384 - acc: 0.9286
Epoch 321/1000
3220/3220 [==============================] - 0s 10us/step - loss: 0.2093 - acc: 0.9444
Epoch 322/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.2288 - acc: 0.9363
Epoch 323/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.2044 - acc: 0.9425
Epoch 324/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1944 - acc: 0.9475
Epoch 325/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1952 - acc: 0.9494
Epo

...

3220/3220 [==============================] - 0s 10us/step - loss: 0.1181 - acc: 0.9686
Epoch 638/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1653 - acc: 0.9519
Epoch 639/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1687 - acc: 0.9512
Epoch 640/1000
3220/3220 [==============================] - 0s 8us/step - loss: 0.1847 - acc: 0.9469
Epoch 641/1000
3220/3220 [==============================] - 0s 8us/step - loss: 0.1813 - acc: 0.9553
Epoch 642/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1666 - acc: 0.9516
Epoch 643/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1447 - acc: 0.9612
Epoch 644/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1508 - acc: 0.9602
Epoch 645/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1519 - acc: 0.9562
Epoch 646/1000
3220/3220 [==============================] - 0s 10us/step - loss: 0.2586 - acc: 0.9525
Epo

3220/3220 [==============================] - 0s 9us/step - loss: 0.1269 - acc: 0.9627
Epoch 719/1000
3220/3220 [==============================] - 0s 10us/step - loss: 0.1629 - acc: 0.9543
Epoch 720/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1497 - acc: 0.9578
Epoch 721/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1481 - acc: 0.9578
Epoch 722/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1441 - acc: 0.9609
Epoch 723/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1595 - acc: 0.9571
Epoch 724/1000
3220/3220 [==============================] - 0s 10us/step - loss: 0.1827 - acc: 0.9488
Epoch 725/1000
3220/3220 [==============================] - 0s 10us/step - loss: 0.0956 - acc: 0.9764
Epoch 726/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1791 - acc: 0.9481
Epoch 727/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1156 - acc: 0.9705
Ep

3220/3220 [==============================] - 0s 9us/step - loss: 0.1848 - acc: 0.9466
Epoch 799/1000
3220/3220 [==============================] - 0s 10us/step - loss: 0.1164 - acc: 0.9705
Epoch 800/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1199 - acc: 0.9696
Epoch 801/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1490 - acc: 0.9562
Epoch 802/1000
3220/3220 [==============================] - 0s 10us/step - loss: 0.1342 - acc: 0.9606
Epoch 803/1000
3220/3220 [==============================] - 0s 10us/step - loss: 0.1962 - acc: 0.9497
Epoch 804/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1113 - acc: 0.9708
Epoch 805/1000
3220/3220 [==============================] - 0s 10us/step - loss: 0.1381 - acc: 0.9593
Epoch 806/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1385 - acc: 0.9612
Epoch 807/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1415 - acc: 0.9581
E

Epoch 879/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.0939 - acc: 0.9730
Epoch 880/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1451 - acc: 0.9599
Epoch 881/1000
3220/3220 [==============================] - 0s 8us/step - loss: 0.1356 - acc: 0.9584
Epoch 882/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1304 - acc: 0.9624
Epoch 883/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1279 - acc: 0.9627
Epoch 884/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1714 - acc: 0.9503
Epoch 885/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1118 - acc: 0.9680
Epoch 886/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1524 - acc: 0.9590
Epoch 887/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1507 - acc: 0.9565
Epoch 888/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1321 - ac

3220/3220 [==============================] - 0s 9us/step - loss: 0.1003 - acc: 0.9699
Epoch 961/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1177 - acc: 0.9714
Epoch 962/1000
3220/3220 [==============================] - 0s 10us/step - loss: 0.1370 - acc: 0.9621
Epoch 963/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1019 - acc: 0.9736
Epoch 964/1000
3220/3220 [==============================] - 0s 10us/step - loss: 0.1844 - acc: 0.9512
Epoch 965/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1031 - acc: 0.9733
Epoch 966/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1383 - acc: 0.9606
Epoch 967/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1207 - acc: 0.9646
Epoch 968/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1366 - acc: 0.9587
Epoch 969/1000
3220/3220 [==============================] - 0s 9us/step - loss: 0.1567 - acc: 0.9581
Epo

## Model Prediction

In [16]:
# Predicting the Test set results. 
#If the output is > 0.5, then label as spam else no spam
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

# Creating the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[488 333]
 [  0 558]]


Now you should have developed a reliable way to measure the accuracy scores,. Feel free to explore different model architectures. (More hidden layers, less ? Layers with more nodes? Less?

## Some concluding remarks

Here is an interesting link that compare other alternatives we might have in building ANN. See [here](https://www.analyticsindiamag.com/pytorch-vs-keras-who-suits-you-the-best/)
With Keras, one can easily use a backend (e.g. Tensorflow), that runs computations on GPUs. Depends on how one would build models,one has to decide (1) level of abstraction, whether using a higher-level API helps (2) define-and-run vs define-by-run. Define-by-run helps develop custom sequence models hence PyTorch, probably still more mature than Tensorflow. On the other hand, Keras is a higher level of abstraction (regardless the backend used) helps when one has some sort of "standard"/"textbook" layers: a fluent API is handy and saves our time otherwise spent on the low-level plumbing.